In [4]:
import pandas as pd
import os
from sqlalchemy import create_engine, text
from sqlalchemy.engine import URL

# Retrieve credentials from environment variables
PG_HOST = os.getenv('PG_HOST')
PG_PORT = int(os.getenv('PG_PORT'))  # Ensure port is an integer
PG_DATABASE = os.getenv('PG_DATABASE')
PG_USER = os.getenv('PG_USER')
PG_PASSWORD = os.getenv('PG_PASSWORD')

# Create the URL object, including the port
url = URL.create(
    drivername="postgresql",
    username=PG_USER,
    password=PG_PASSWORD,
    host=PG_HOST,
    port=PG_PORT,
    database=PG_DATABASE
)

# Create the engine
engine = create_engine(url)


In [5]:
# Define the query using text()
query = text("""SELECT DISTINCT
    p.SUBJECT_ID,
    p.GENDER,
    p.DOB,
    p.DOD,
    p.EXPIRE_FLAG,
    a.HADM_ID,
    a.ADMITTIME,
    a.DISCHTIME,
    a.ADMISSION_TYPE,
    EXTRACT(YEAR FROM AGE(a.ADMITTIME, p.DOB)) AS AGE_AT_ADMISSION
FROM
    mimiciii.patients p
JOIN
    mimiciii.diagnoses_icd d
    ON p.SUBJECT_ID = d.SUBJECT_ID
JOIN
    mimiciii.admissions a
    ON d.HADM_ID = a.HADM_ID
WHERE
    d.ICD9_CODE IN ('99591', '99592');
""")

# Query and load data into a DataFrame using a context manager for connection
with engine.connect() as connection:
    df = pd.read_sql_query(query, connection)

# Show the first few rows of the DataFrame
df.head()

,subject_id,gender,dob,dod,expire_flag,hadm_id,admittime,dischtime,admission_type,age_at_admission
0,88081,F,2063-11-20,2138-10-29,1,113344,2138-09-12 20:44:00,2138-10-29 19:50:00,EMERGENCY,74.0
1,17112,F,2027-08-04,2105-05-17,1,121197,2105-01-10 12:34:00,2105-01-16 16:50:00,EMERGENCY,77.0
2,48453,M,2143-06-15,2201-01-11,1,183977,2200-11-15 22:15:00,2200-12-22 16:22:00,EMERGENCY,57.0
3,18471,M,2099-08-14,2175-07-23,1,150976,2175-07-14 02:36:00,2175-07-23 02:30:00,EMERGENCY,75.0
4,20385,F,2085-12-02,NaT,0,177271,2167-03-28 05:11:00,2167-04-10 16:15:00,EMERGENCY,81.0
